In [23]:
!pip install requests beautifulsoup4 pandas
!pip install atlasclient



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'atlasclient' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'atlasclient'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for atlasclient: filename=atlasclient-1.0.0-py3-none-any.whl size=29174 sha256=ab13d3076d4e94f0a62a8bdca566f4c49c4b5ce199520a6b4bbe41b2b901c421
  Stored in directory: /Users/vaibhavpandey/Library/Caches/pip/wheels/b2/77/c6/94d6fd4ca781735e546114b844f6f5a25d592664fd305e9096
Successfully built atlasclient

[notice] A new release of pip is available: 25.2 -> 25.3
[noti

In [ ]:
import pandas as pd

df_srs = pd.read_csv("download_samples_tsv-2.tsv", sep="\t", dtype=str)
df_srs = df_srs.rename(columns={"#sid": "SRS"})
df_srs["SRS"] = df_srs["SRS"].astype(str)

df_meta = pd.read_csv("metadata.csv", dtype=str)

print("SRS columns:", df_srs.columns.tolist())
print("Meta columns:", df_meta.columns.tolist())

SRS columns: ['SRS', 'name', 'note', 'sample_env', 'keywords_clean', 'taxa_stats', 'num_rids', 'num_hq_runs', 'rids', 'projects', 'publications']
Meta columns: ['subjectID', 'SampleID', 'age_at_collection', 'collection_month', 'delivery', 'gest_time', 'gender', 'country', 'Exclusive_breast_feeding', 'Breast_feeding_end', 'Regular_formula', 'hydrosylated_formula', 'partly_hydrosylated_formula', 'Any_baby_formula', 'Fruits_and_berries', 'Corn', 'Rice', 'Wheat', 'Oat', 'Barley', 'Rye', 'Cereal', 'Other_grains', 'Root_vegetables', 'Vegetables', 'Eggs', 'Soy', 'Milk', 'Meat', 'Fish', 'Other_food', 'Other_than_BF', 'bf_length', 'num_abx_treatments', 'num_abx_first_year', 'abx_first_year', 'after_abx', 'num_preceeding_abx', 'hla_risk_class', 'seroconverted', 'num_aabs', 'totalige', 'totalige_log', 'allergy_milk', 'allergy_egg', 'allergy_peanut', 'allergy_dustmite', 'totalige_high', 'allergy_cat', 'allergy_dog', 'allergy_birch', 'allergy_timothy', 'gid_wgs', 'mgx_reads', 'mgx_pool', 'mgx_reads

In [66]:
import requests
import re

def get_host_subject_id(srs_id: str) -> str | None:
    """
    Use ENA browser XML API to get host_subject_id for a given SRS accession.
    This matches the 'Original MetaData' you see in MicrobeAtlas.
    """
    url = f"https://www.ebi.ac.uk/ena/browser/api/xml/{srs_id}"
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
    except Exception as e:
        print(f"[ERROR] SRS={srs_id}: {e}")
        return None

    xml = r.text

    # ENA attribute structure is like:
    # <SAMPLE_ATTRIBUTE>
    #   <TAG>host_subject_id</TAG>
    #   <VALUE>P018832</VALUE>
    # </SAMPLE_ATTRIBUTE>
    m = re.search(
        r"<TAG>\s*host_subject_id\s*</TAG>\s*<VALUE>\s*([^<\s]+)\s*</VALUE>",
        xml,
        flags=re.IGNORECASE,
    )
    if m:
        return m.group(1)

    print(f"[WARN] no host_subject_id found in XML for {srs_id}")
    return None

In [67]:
test_srs = "SRS1719087"
print("Testing SRS:", test_srs)
print("host_subject_id:", get_host_subject_id(test_srs))

Testing SRS: SRS1719087
host_subject_id: P018832


In [ ]:
import pandas as pd
import time

df_srs = pd.read_csv("download_samples_tsv-2.tsv", sep="\t", dtype=str)
df_srs = df_srs.rename(columns={"#sid": "SRS"})
df_srs["SRS"] = df_srs["SRS"].astype(str)

unique_srs = df_srs["SRS"].dropna().unique().tolist()
print("Number of unique SRS:", len(unique_srs))

rows = []
for i, srs in enumerate(unique_srs, start=1):
    subj = get_host_subject_id(srs)
    rows.append({"SRS": srs, "host_subject_id": subj})

    if i % 50 == 0:
        print(f"{i}/{len(unique_srs)} SRS processed")
        time.sleep(0.2)

df_map = pd.DataFrame(rows)
df_map.to_csv("srs_to_host_subject_id.csv", index=False)
df_map.head()

Number of unique SRS: 785
50/785 SRS processed
100/785 SRS processed
150/785 SRS processed
200/785 SRS processed
250/785 SRS processed
300/785 SRS processed
350/785 SRS processed
400/785 SRS processed
450/785 SRS processed
500/785 SRS processed
550/785 SRS processed
600/785 SRS processed
650/785 SRS processed
700/785 SRS processed
750/785 SRS processed


,SRS,host_subject_id
0,SRS1719087,P018832
1,SRS1719088,P017743
2,SRS1719089,P000648
3,SRS1719090,T022883
4,SRS1719091,T012374


In [ ]:
df_meta = pd.read_csv("metadata.csv", dtype=str)

subj_col = None
for c in df_meta.columns:
    if c.lower() in ("subjectid", "subject_id", "host_subject_id", "host_subjectid"):
        subj_col = c
        break

print("Detected subject column:", subj_col)
if subj_col is None:
    raise ValueError(f"Could not find subjectID column in metadata. Columns: {df_meta.columns.tolist()}")

df_meta = df_meta.rename(columns={subj_col: "subjectID"})
df_map = df_map.rename(columns={"host_subject_id": "subjectID"})

df_merged = df_map.merge(df_meta, on="subjectID", how="left")
print("Merged columns:", df_merged.columns.tolist())
df_merged.head()

Detected subject column: subjectID
Merged columns: ['SRS', 'subjectID', 'SampleID', 'age_at_collection', 'collection_month', 'delivery', 'gest_time', 'gender', 'country', 'Exclusive_breast_feeding', 'Breast_feeding_end', 'Regular_formula', 'hydrosylated_formula', 'partly_hydrosylated_formula', 'Any_baby_formula', 'Fruits_and_berries', 'Corn', 'Rice', 'Wheat', 'Oat', 'Barley', 'Rye', 'Cereal', 'Other_grains', 'Root_vegetables', 'Vegetables', 'Eggs', 'Soy', 'Milk', 'Meat', 'Fish', 'Other_food', 'Other_than_BF', 'bf_length', 'num_abx_treatments', 'num_abx_first_year', 'abx_first_year', 'after_abx', 'num_preceeding_abx', 'hla_risk_class', 'seroconverted', 'num_aabs', 'totalige', 'totalige_log', 'allergy_milk', 'allergy_egg', 'allergy_peanut', 'allergy_dustmite', 'totalige_high', 'allergy_cat', 'allergy_dog', 'allergy_birch', 'allergy_timothy', 'gid_wgs', 'mgx_reads', 'mgx_pool', 'mgx_reads_filtered', 'read_count_16S', 'sequencing_PDO_16S', 'gid_16s']


,SRS,subjectID,SampleID,age_at_collection,collection_month,delivery,gest_time,gender,country,Exclusive_breast_feeding,...,allergy_dog,allergy_birch,allergy_timothy,gid_wgs,mgx_reads,mgx_pool,mgx_reads_filtered,read_count_16S,sequencing_PDO_16S,gid_16s
0,SRS1719087,P018832,3104160,597.0,20.0,vaginal,278.0,Female,RUS,False,...,False,False,False,G80333,25397552.0,Plate 8,25.322641,20683.0,PDO-4156,G73770
1,SRS1719087,P018832,3100410,292.0,10.0,vaginal,278.0,Female,RUS,False,...,False,False,False,G78751,26226378.0,Plate 2,26.156407,19873.0,PDO-3146,G65936
2,SRS1719087,P018832,3100412,358.0,12.0,vaginal,278.0,Female,RUS,False,...,False,False,False,G69201,12784726.0,old_batch,12.777236,46515.0,PDO-4098,G65938
3,SRS1719087,P018832,3104039,445.0,15.0,vaginal,278.0,Female,RUS,False,...,False,False,False,NaN,NaN,NaN,NaN,101199.0,PDO-4156,G73640
4,SRS1719087,P018832,3104038,416.0,14.0,vaginal,278.0,Female,RUS,False,...,False,False,False,G78566,26477490.0,Plate 3,26.366903,32674.0,PDO-3108,G64586


In [74]:
allergy_cols = [
    "allergy_milk",
    "allergy_egg",
    "allergy_peanut",
    "allergy_dustmite",
    "allergy_cat",
    "allergy_dog",
    "allergy_birch",
    "allergy_timothy",
    "totalige_high",
]

missing = [c for c in allergy_cols if c not in df_merged.columns]
if missing:
    raise ValueError(f"Missing allergy columns in df_merged: {missing}")

def any_allergy(row):
    for col in allergy_cols:
        v = str(row[col]).strip().lower()
        if v in {"1", "true", "yes"}:
            return 1
    return 0

df_merged["label"] = df_merged.apply(any_allergy, axis=1)

df_merged[["SRS", "subjectID", "collection_month", "label"]].head()

,SRS,subjectID,collection_month,label
0,SRS1719087,P018832,20.0,1
1,SRS1719087,P018832,10.0,1
2,SRS1719087,P018832,12.0,1
3,SRS1719087,P018832,15.0,1
4,SRS1719087,P018832,14.0,1


In [75]:
df_merged = df_merged.dropna(subset=["collection_month"]).copy()
df_merged["collection_month"] = (
    pd.to_numeric(df_merged["collection_month"], errors="coerce")
    .round()
    .astype(int)
)

agg = (
    df_merged
    .groupby(["SRS", "subjectID", "collection_month"], as_index=False)
    .agg(label=("label", "max")) 
)

agg.head()

,SRS,subjectID,collection_month,label
0,SRS1719087,P018832,6,1
1,SRS1719087,P018832,7,1
2,SRS1719087,P018832,10,1
3,SRS1719087,P018832,11,1
4,SRS1719087,P018832,12,1


In [76]:
agg.to_csv("ai_allergies_srs_labels_all_months.csv", index=False)
print("Master labeled file written: ai_allergies_srs_labels_all_months.csv")
print(agg.head())

Master labeled file written: ai_allergies_srs_labels_all_months.csv
          SRS subjectID  collection_month  label
0  SRS1719087   P018832                 6      1
1  SRS1719087   P018832                 7      1
2  SRS1719087   P018832                10      1
3  SRS1719087   P018832                11      1
4  SRS1719087   P018832                12      1


In [77]:
for m, grp in agg.groupby("collection_month"):
    out = grp[["SRS", "label"]].drop_duplicates().copy()
    out = out.rename(columns={"SRS": "sample_id"})
    fname = f"Month_{m}.csv"
    out.to_csv(fname, index=False)
    print(f"Wrote {fname} with {len(out)} rows")

Wrote Month_1.csv with 155 rows
Wrote Month_2.csv with 233 rows
Wrote Month_3.csv with 99 rows
Wrote Month_4.csv with 410 rows
Wrote Month_5.csv with 121 rows
Wrote Month_6.csv with 191 rows
Wrote Month_7.csv with 536 rows
Wrote Month_8.csv with 219 rows
Wrote Month_9.csv with 208 rows
Wrote Month_10.csv with 569 rows
Wrote Month_11.csv with 274 rows
Wrote Month_12.csv with 218 rows
Wrote Month_13.csv with 560 rows
Wrote Month_14.csv with 252 rows
Wrote Month_15.csv with 187 rows
Wrote Month_16.csv with 598 rows
Wrote Month_17.csv with 219 rows
Wrote Month_18.csv with 252 rows
Wrote Month_19.csv with 487 rows
Wrote Month_20.csv with 209 rows
Wrote Month_21.csv with 148 rows
Wrote Month_22.csv with 463 rows
Wrote Month_23.csv with 227 rows
Wrote Month_24.csv with 146 rows
Wrote Month_25.csv with 78 rows
Wrote Month_26.csv with 72 rows
Wrote Month_27.csv with 115 rows
Wrote Month_28.csv with 361 rows
Wrote Month_29.csv with 80 rows
Wrote Month_30.csv with 22 rows
Wrote Month_31.csv with 